# Hybrid Model Performance Comparison

This notebook compares the performance of:
1. Original CNN-BiLSTM model
2. Improved CNN-BiLSTM architecture
3. Analysis of improvements in accuracy and efficiency

In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, confusion_matrix

# Load test data
X_test = np.load('preprocessed/X_test.npy')
y_test = np.load('preprocessed/y_test.npy')
X_test_reshaped = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

# Load both models
original_model = load_model('models/CNN-BiLSTM.h5')
improved_model = load_model('models/improved_cnn_bilstm.h5')

In [ ]:
def evaluate_model(model, name):
    """Evaluate model and return metrics"""
    # Predictions
    y_pred = np.argmax(model.predict(X_test_reshaped), axis=1)
    
    # Calculate metrics
    report = classification_report(y_test, y_pred, output_dict=True)
    cm = confusion_matrix(y_test, y_pred)
    
    # Plot confusion matrix
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Confusion Matrix - {name}')
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.show()
    
    return report

# Evaluate both models
print("Original Model Performance:")
original_metrics = evaluate_model(original_model, "Original CNN-BiLSTM")

print("\nImproved Model Performance:")
improved_metrics = evaluate_model(improved_model, "Improved CNN-BiLSTM")

In [ ]:
# Compare model architectures
print("Model Architecture Comparison:")
print("\nOriginal Model:")
original_model.summary()

print("\nImproved Model:")
improved_model.summary()

# Compare parameters and size
original_params = original_model.count_params()
improved_params = improved_model.count_params()

print(f"\nParameter Count:")
print(f"Original Model: {original_params:,}")
print(f"Improved Model: {improved_params:,}")
print(f"Difference: {improved_params - original_params:,}")

In [ ]:
# Plot accuracy comparison
metrics = {
    'Model': ['Original', 'Improved'],
    'Accuracy': [original_metrics['accuracy'], improved_metrics['accuracy']],
    'Macro F1': [original_metrics['macro avg']['f1-score'], 
                 improved_metrics['macro avg']['f1-score']]
}

plt.figure(figsize=(10, 6))
x = np.arange(len(metrics['Model']))
width = 0.35

plt.bar(x - width/2, metrics['Accuracy'], width, label='Accuracy')
plt.bar(x + width/2, metrics['Macro F1'], width, label='Macro F1')

plt.xlabel('Model Version')
plt.ylabel('Score')
plt.title('Model Performance Comparison')
plt.xticks(x, metrics['Model'])
plt.legend()

# Add value labels
for i in x:
    plt.text(i - width/2, metrics['Accuracy'][i], f"{metrics['Accuracy'][i]:.4f}", 
             ha='center', va='bottom')
    plt.text(i + width/2, metrics['Macro F1'][i], f"{metrics['Macro F1'][i]:.4f}", 
             ha='center', va='bottom')

plt.tight_layout()
plt.show()

In [ ]:
# Print detailed comparison
print("Detailed Performance Comparison:")
print("\nOriginal Model:")
print(f"Accuracy: {original_metrics['accuracy']:.4f}")
print(f"Macro F1: {original_metrics['macro avg']['f1-score']:.4f}")
print(f"Weighted F1: {original_metrics['weighted avg']['f1-score']:.4f}")

print("\nImproved Model:")
print(f"Accuracy: {improved_metrics['accuracy']:.4f}")
print(f"Macro F1: {improved_metrics['macro avg']['f1-score']:.4f}")
print(f"Weighted F1: {improved_metrics['weighted avg']['f1-score']:.4f}")

# Calculate improvements
acc_improvement = (improved_metrics['accuracy'] - original_metrics['accuracy']) * 100
f1_improvement = (improved_metrics['macro avg']['f1-score'] - 
                 original_metrics['macro avg']['f1-score']) * 100

print(f"\nImprovements:")
print(f"Accuracy: +{acc_improvement:.2f}%")
print(f"Macro F1: +{f1_improvement:.2f}%")